<a href="https://colab.research.google.com/github/socd06/databricks-language-hackathon/blob/main/June6_Dolly_3B_Building_Code_Expert_PDF_Question_Answering_with_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
!pip install transformers accelerate einops Xformers langchain InstructorEmbedding sentence-transformers chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install unstructured pandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 63.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 

In [ ]:
# CLEAR CUDA
import gc
torch.cuda.empty_cache()
gc.collect()


In [3]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer

model_name = "databricks/dolly-v2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

generate_text = pipeline(model=model_name, torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)

In [ ]:
hf_pipeline

# Test LLM Chain

In [5]:
question = 'Who was Dolly the sheep?'
llm_chain.run(question)

'\nDolly the sheep was an animal that was the first successful human embryo to be created by in vitro fertilization, or in other words, creating a human embryo in a lab. Dolly was created in 1997 by American biologist Patrick McEwan, who inserted the embryo into a sheep, which then gave birth to Dolly. Dolly is also the name of the sheep.'

In [6]:
question = 'what are the colors of the rainbow?'
llm_chain.run(question)

'\nThe colors of the rainbow are Pinks, Oranges, Yellows, Turquoises, Indigos and Blues.'

# Prepare Documents


In [7]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install pypdf


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 3.3 MB/s eta 0:00:00


In [88]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader

In [94]:
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Download the National Building Code
from [this link](https://nrc-publications.canada.ca/eng/view/object/?id=515340b5-f4e0-4798-be69-692e4ec423e8) and put in the 'docs' folder

In [107]:
# Load and process the PDF files

# faster, but not quite good performance
loader = DirectoryLoader('/content/docs/', glob="./*.pdf", loader_cls=PyPDFLoader)

# Way slower, hopefully has better performance
# loader = UnstructuredPDFLoader('/content/docs/nbc2020_p1.pdf', mode='elements')
documents = loader.load()

In [160]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
texts = text_splitter.split_documents(documents)

In [161]:
len(texts)

7257

In [162]:
texts[2]

Document(page_content='First Edition 1941\nSecond Edition 1953\nThird Edition 1960\nFourth Edition 1965\nFifth Edition 1970\nSixth Edition 1975\nSeventh Edition 1977\nEighth Edition 1980\nNinth Edition 1985\nTenth Edition 1990\nEleventh Edition 1995\nTwelfth Edition 2005\nThirteenth Edition 2010\nFourteenth Edition 2015\nFifteenth Edition 2020\nPaper: ISBN 978-0-660-37913-5 NR24-28/2020E\nPDF: ISBN 978-0-660-37912-8 NR24-28/2020E-PDF\nNRCC-CONST-56435E\n© His Majesty the King in Right of Canada, \nas represented by the National Research \nCouncil of Canada, 2022\nPrinted\n in Canada\nFirst Printing\nAussi disponible en français :\nCode national du bâtiment – Canada 2020\nNRCC-CONST-56435F\nPapier : ISBN 978-0-660-37915-9\nPDF : ISBN 978-0-660-37914-2\n© His Majesty the King in Right of Canada, as represented by the National Research Council of Canada, 2022 © Sa Majesté le Roi du chef du Canada, représenté par le Conseil national de recherches du Canada, 2022', metadata={'source': '/con

# Download HuggingFace Embeddings
Check [MTEB English Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) to make sure you download embeddings with good performance

In [16]:
# Choose one of the top performers from the MTEB English Leaderboard

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# top #2 when task = Retrieval June 2023 for under ~500 MB
model_name = "intfloat/e5-base-v2" 

hf = HuggingFaceEmbeddings(model_name=model_name)

# Make a Vector Database

In [163]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the new embeddings being used
embedding = hf 

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# and a retriever

In [164]:
retriever = vectordb.as_retriever(search_kwargs={'k':3})

# Test the basic retriever

In [180]:
docs = retriever.get_relevant_documents("What are the minimum ceiling height requirements for habitable rooms?")

In [181]:
for doc in docs:
  print(doc)

page_content='6000 to 6999Zone 8\n≥7 0 0 0Above-ground Opaque Building\nAssembly\nMinimum Effective Thermal Resistance (RSI), (m2×K)/W\nCeilings below attics 6.91 6.91 8.67 8.67 10.43 10.43\nCathedral ceilings and flat roofs 4.67 4.67 4.67 5.02 5.02 5.02\nWalls(2) 2.78 2.97 2.97 2.97 3.08 3.08\nFloors over unheated spaces 4.67 4.67 4.67 5.02 5.02 5.02\nNotes to Table 9.36.2.6.-B:\n(1)See Article 1.1.3.1.\n(2)See Sentence 9.36.2.8.(3) for requirements concerning the above-ground portion of foundation walls.\n2)The effective thermal resistance of rim joists shall be not less than that required\nfor above-ground walls in Table 9.36.2. 6.-A or 9.36.2.6.-B, as applicable.\n3)A reduction in the effective thermal resistance of ceiling assemblies in attics\nunder sloped roofs is permitted for a length no greater than 1 200 mm but only to\nthe extent imposed by the roof slope and minimum venting clearance, provided thenominal thermal resistance of the insulation directly above the exterior wall

# Make a proper Question Retrieval chain

In [174]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
      print(source.metadata['source'] + ' Page ' + str(source.metadata['page']))

In [175]:
from langchain.memory import ConversationBufferMemory

from typing import Dict, Any

class AnswerConversationBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerConversationBufferMemory, self).save_context(inputs,{'response': outputs['result']})

In [176]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                                   chain_type="stuff", 
                                                   retriever=retriever, 
                                                   return_source_documents=True,
                                                   memory=AnswerConversationBufferMemory())

In [183]:
# try to set the tone
template = '''
You are the helper of a tradesperson with knowledge of the Canadian Building Code. You provide specific details using the context given and the users question. 
If you don't know the answer, you truthfully say you don't know and don't try to make up an answer. 
----------------
{context}

Question: {question}
Helpful Answer:'''

In [184]:
qa_chain_with_memory.combine_documents_chain.llm_chain.prompt.template = template
qa_chain_with_memory.combine_documents_chain.llm_chain.prompt.template

"\nYou are the helper of a tradesperson with knowledge of the Canadian Building Code. You provide specific details using the context given and the users question. \nIf you don't know the answer, you truthfully say you don't know and don't try to make up an answer. \n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [185]:
query = "What are the minimum ceiling height requirements for habitable rooms?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)


Rise to a minimum of 2.96m above grade.


Sources:
/content/docs/nbc2020_p1.pdf Page 1056
/content/docs/nbc2020_p1.pdf Page 1336
/content/docs/nbc2020_p1.pdf Page 398


In [186]:
query = "Are there specific guidelines in the National Building Code regarding the insulation requirements for exterior walls in residential buildings?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)


No. The National Building Code does not provide separate requirements which would apply to seasonally or intermittently occupied buildings.


Sources:
/content/docs/nbc2020_p1.pdf Page 1326
/content/docs/nbc2020_p1.pdf Page 979
/content/docs/nbc2020_p1.pdf Page 1468


Here we see that it sent us to another document instead of trying to come up with an answer. Exactly the behavior we want!

In [187]:
query = "Can you provide information on the fire safety regulations for commercial buildings?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)


A commercial building is defined under the Canadian Building Code as "any indoor space used for trade, business, entertainment, or residence purposes". In addition to the Canadian Building Code, most provinces and states in Canada have their own building codes that apply. Therefore, in order to provide specific information on the requirements and regulations for commercial buildings, it is first necessary to be aware of the specific requirements of your province or state.

Fire safety regulations for commercial buildings will often include requirements for fire separations, such as the use of different areas for different purposes (e.g. areas for assembly and areas for activities such as eating), design of barrier-free floors, and the use of fire-retardant materials in construction.  Some provinces and states require the use of approved approved fire-retardant materials for construction of commercial buildings.  Some common approved fire-retardant materials are:       

-  Coniferous 

In [189]:
query = "What are the accessibility requirements for ramps and handrails in public buildings?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)


Public buildings are typically designed with ramps and handrails in place for individuals with disabilities. The criteria for ramps and handrails are provided in the Accessibility for Ontarians with Disabilities Act, 2016 (ONBAD). 

As an example, in buildings with barrier-free pedestrian pathways for people with disabilities, intermediate handrails shall be provided so that: 
- a handrail is reachable within 750mm of all portions of the required exit width, 
- at least one portion of the stair or ramp between two handrails is the minimum width required for stairways or ramps. 

Intermediate handrails may be located along the most direct path of travel or may be continuously graspable along their entire length.


Sources:
/content/docs/nbc2020_p1.pdf Page 317
/content/docs/nbc2020_p1.pdf Page 288
/content/docs/nbc2020_p1.pdf Page 274


# Trying out Knowledge Graph Memory
# NEEDS FURTHER TESTING

In [ ]:
# workaround again
class AnswerKnowledgeGraphMemory(ConversationKGMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerKnowledgeGraphMemory, self).save_context(inputs,{'response': outputs['result']})

In [ ]:
from langchain.memory import ConversationKGMemory

memory = AnswerKnowledgeGraphMemory(llm=hf_pipeline)

qa_chain_with_kg_memory = RetrievalQA.from_chain_type(llm=hf_pipeline,
                                                      chain_type="stuff", 
                                                      retriever=retriever, 
                                                      return_source_documents=True,                                                      
                                                      memory=memory)

In [ ]:
# try to set the tone to "as a human-like Student Advisor"
template = '''
You are a prospective student advisor for a university. You are talkative and provide lots of specific details using the context given by the users question. 
If you don't know the answer, you truthfully say you don't know and don't try to make up an answer. You use the information contained in the "Relevant Information" section and You always answer with unbiased, ETHICAL and safe advise.

Relevant Information:
{history}

Conversation:

Question: {question}
Helpful Answer:'''

prompt = PromptTemplate(
    input_variables=["history", "question"], template=template)

In [ ]:
qa_chain_with_kg_memory.combine_documents_chain.llm_chain.prompt = prompt
qa_chain_with_kg_memory.combine_documents_chain.llm_chain.prompt

In [ ]:
qa_chain_with_kg_memory.combine_documents_chain

In [ ]:
query = "I am interested in both medicine and engineering, do you have a program that is a mix of the two?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "I like biomedical engineering, do you offer it as a graduate program?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "How much is tuition for this program?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "WOw, expensive! is this the fee for international students?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

# Notes for next version
- Keep overlap = 300
- Make an Ontario Building Code version (see [this link](https://www.buildingcode.online)) and [sitemap link](https://www.buildingcode.online/sitemap.xml)
- DO NOT Use dolly tokenizer to make vector database ([Reference link](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/huggingface_length_function.html)). It ends up giving mismatching tensor errors.

- Figure out how to handle data in databricks using dbfs ([Dolly Example](https://www.dbdemos.ai/demo-notebooks.html?demoName=llm-dolly-chatbot))
- Databricks can do online versions instead of dealing with dbfs!